# Part X: Advanced Topics and Ecosystem

## Chapter 23: GraphQL

While REST APIs excel at resource-based operations, GraphQL offers a powerful alternative for complex data fetching requirements. It allows clients to request exactly the data they need, reducing over-fetching and under-fetching. This chapter covers GraphQL fundamentals, integration with FastAPI using Strawberry (the modern Python GraphQL library), and advanced patterns for production applications.

---

### 23.1 Introduction to GraphQL: Understanding the Query Language

GraphQL is a query language for APIs and a runtime for executing those queries against your data. Unlike REST, where endpoints return fixed data structures, GraphQL exposes a single endpoint that accepts flexible queries.

#### GraphQL vs REST Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                    GraphQL vs REST Comparison                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  REST API: Multiple Endpoints, Fixed Responses                   │
│  ─────────────────────────────────────────────────────────────  │
│                                                                  │
│  GET /users/1          → { id, name, email, phone, address }    │
│  GET /users/1/posts    → [ { id, title }, { id, title } ]       │
│  GET /users/1/friends  → [ { id, name }, { id, name } ]          │
│                                                                  │
│  Problems:                                                       │
│  • Over-fetching: Got phone/address when only needed name      │
│  • Under-fetching: Need separate request for posts + friends    │
│  • N+1 queries: Each friend requires another request for details   │
│                                                                  │
│  ─────────────────────────────────────────────────────────────  │
│                                                                  │
│  GraphQL API: Single Endpoint, Flexible Queries                    │
│  ─────────────────────────────────────────────────────────────  │
│                                                                  │
│  POST /graphql                                                   │
│  {                                                               │
│    query {                                                       │
│      user(id: 1) {                                               │
│        name                                                      │
│        posts { title, createdAt }                                 │
│        friends {                                                 │
│          name                                                    │
│          posts(limit: 5) { title }                               │
│        }                                                         │
│      }                                                           │
│    }                                                             │
│  }                                                               │
│                                                                  │
│  Response: Exactly what was requested, nothing more            │
│  {                                                               │
│    "data": {                                                     │
│      "user": {                                                   │
│        "name": "Alice",                                          │
│        "posts": [ { "title": "Hello", "createdAt": "..." } ],   │
│        "friends": [                                              │
│          { "name": "Bob", "posts": [...] },                      │
│          { "name": "Carol", "posts": [...] }                    │
│        ]                                                         │
│      }                                                           │
│    }                                                             │
│  }                                                               │
│                                                                  │
│  Benefits:                                                       │
│  • Single request for complex, nested data                       │
│  • No over-fetching (client specifies fields)                    │
│  • Strongly typed schema (introspection, auto-docs)              │
│  • Real-time subscriptions built-in                              │
│                                                                  │
│  Trade-offs:                                                     │
│  • Caching is more complex (no HTTP semantics)                   │
│  • File uploads require multipart spec                           │
│  • Query complexity can abuse server (depth limiting needed)     │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

#### Core GraphQL Concepts

```python
# graphql_concepts.py - GraphQL type system explained

"""
GraphQL Schema Definition:

1. Types: Define the shape of data
   - Object types: User, Post (complex objects)
   - Scalar types: String, Int, Float, Boolean, ID (primitives)
   - Enum types: Role (ADMIN, USER, GUEST)
   - Interface types: Node (shared fields across types)
   - Union types: SearchResult (User | Post | Comment)

2. Operations:
   - Query: Read operations (GET equivalent)
   - Mutation: Write operations (POST/PUT/DELETE equivalent)
   - Subscription: Real-time updates (WebSocket)

3. Schema: Collection of types and operations
"""

# Example GraphQL Schema (SDL - Schema Definition Language)
"""
type User {
  id: ID!
  username: String!
  email: String!
  posts: [Post!]!
  friends: [User!]!
  createdAt: DateTime!
}

type Post {
  id: ID!
  title: String!
  content: String!
  author: User!
  published: Boolean!
  tags: [String!]!
}

type Query {
  # Single item by ID
  user(id: ID!): User
  
  # List with filtering/pagination
  users(
    limit: Int = 10
    offset: Int = 0
    search: String
  ): [User!]!
  
  # Current user
  me: User!
}

type Mutation {
  createUser(input: CreateUserInput!): User!
  updateUser(id: ID!, input: UpdateUserInput!): User!
  deleteUser(id: ID!): Boolean!
}

type Subscription {
  userCreated: User!
  postPublished(authorId: ID!): Post!
}

input CreateUserInput {
  username: String!
  email: String!
  password: String!
}

# ! means non-nullable
# [User!]! means non-null list of non-null Users
"""
```

---

### 23.2 Integrating Strawberry: Code-First GraphQL

Strawberry is a modern Python GraphQL library that uses Python type hints for schema definition. It integrates seamlessly with FastAPI and supports async/await natively.

#### Setting Up Strawberry with FastAPI

```python
# strawberry_setup.py - Basic integration
import strawberry
from strawberry.fastapi import GraphQLRouter
from fastapi import FastAPI, Depends
from typing import List, Optional
import uuid
from datetime import datetime

# ═════════════════════════════════════════════════════════════════
# Schema Definition with Strawberry
# ═════════════════════════════════════════════════════════════════

@strawberry.type
class User:
    """GraphQL User type."""
    id: strawberry.ID
    username: str
    email: str
    created_at: datetime
    
    # Resolver for computed field
    @strawberry.field
    def display_name(self) -> str:
        """Computed field - not stored in DB."""
        return f"@{self.username}"
    
    # Resolver for relationship
    @strawberry.field
    async def posts(self, info) -> List["Post"]:
        """Lazy-loaded relationship."""
        # Access database from context
        db = info.context["db"]
        return await get_user_posts(db, self.id)


@strawberry.type
class Post:
    """GraphQL Post type."""
    id: strawberry.ID
    title: str
    content: str
    published: bool
    created_at: datetime
    author_id: strawberry.ID
    
    @strawberry.field
    async def author(self, info) -> User:
        """Resolve author from author_id."""
        db = info.context["db"]
        return await get_user_by_id(db, self.author_id)


# Input types for mutations
@strawberry.input
class CreateUserInput:
    username: str
    email: str
    password: str


@strawberry.input
class UpdateUserInput:
    username: Optional[str] = None
    email: Optional[str] = None


# ═════════════════════════════════════════════════════════════════
# Query Definition
# ═════════════════════════════════════════════════════════════════

@strawberry.type
class Query:
    @strawberry.field
    async def user(self, info, id: strawberry.ID) -> Optional[User]:
        """Get user by ID."""
        db = info.context["db"]
        return await get_user_by_id(db, id)
    
    @strawberry.field
    async def users(
        self,
        info,
        limit: int = 10,
        offset: int = 0,
        search: Optional[str] = None
    ) -> List[User]:
        """List users with pagination."""
        db = info.context["db"]
        return await list_users(db, limit=limit, offset=offset, search=search)
    
    @strawberry.field
    async def me(self, info) -> User:
        """Get current user."""
        user_id = info.context.get("user_id")
        if not user_id:
            raise Exception("Not authenticated")
        db = info.context["db"]
        return await get_user_by_id(db, user_id)


# ═════════════════════════════════════════════════════════════════
# Mutation Definition
# ═════════════════════════════════════════════════════════════════

@strawberry.type
class Mutation:
    @strawberry.mutation
    async def create_user(self, info, input: CreateUserInput) -> User:
        """Create new user."""
        db = info.context["db"]
        
        # Validate
        if await user_exists(db, input.username):
            raise Exception(f"Username {input.username} taken")
        
        # Create
        user = await create_user_db(
            db,
            username=input.username,
            email=input.email,
            password=hash_password(input.password)
        )
        
        return User(
            id=str(user.id),
            username=user.username,
            email=user.email,
            created_at=user.created_at
        )
    
    @strawberry.mutation
    async def update_user(
        self,
        info,
        id: strawberry.ID,
        input: UpdateUserInput
    ) -> User:
        """Update user."""
        db = info.context["db"]
        user = await get_user_by_id(db, id)
        if not user:
            raise Exception("User not found")
        
        # Update fields
        if input.username:
            user.username = input.username
        if input.email:
            user.email = input.email
        
        await db.commit()
        return user
    
    @strawberry.mutation
    async def delete_user(self, info, id: strawberry.ID) -> bool:
        """Delete user."""
        db = info.context["db"]
        await delete_user_db(db, id)
        return True


# ═════════════════════════════════════════════════════════════════
# FastAPI Integration
# ═════════════════════════════════════════════════════════════════

# Create schema
schema = strawberry.Schema(query=Query, mutation=Mutation)

# Create FastAPI app
app = FastAPI()

# GraphQL context dependency
async def get_graphql_context():
    """Provide database and auth context to GraphQL resolvers."""
    async with AsyncSessionLocal() as db:
        # In real app, extract user from JWT
        user_id = "extract-from-jwt"
        
        yield {
            "db": db,
            "user_id": user_id,
            "request_time": datetime.utcnow()
        }

# Add GraphQL router
app.include_router(
    GraphQLRouter(
        schema,
        context_getter=get_graphql_context,
        graphql_ide="apollo-sandbox",  # or "graphiql"
    ),
    prefix="/graphql"
)

# Optional: REST endpoints alongside GraphQL
@app.get("/health")
async def health():
    return {"status": "ok"}
```

**Key Strawberry Concepts:**

1. **Decorators**: `@strawberry.type` for output types, `@strawberry.input` for input types, `@strawberry.field` for resolvers
2. **Context**: `info.context` dictionary passed to all resolvers for dependencies (database, auth)
3. **Resolvers**: Methods that fetch data. Can be async and access context
4. **IDE**: Built-in GraphQL Playground or Apollo Sandbox for testing queries

---

### 23.3 Advanced Patterns: DataLoaders, Authentication, and Subscriptions

Production GraphQL requires solving the N+1 query problem, handling authentication, and supporting real-time subscriptions.

#### DataLoader Pattern (N+1 Prevention)

```python
# dataloader.py - Solving N+1 queries
from strawberry.dataloader import DataLoader
from typing import List
from collections import defaultdict

"""
The N+1 Problem in GraphQL:

Query:
{
  users {
    name
    posts { title }  # Each user triggers a posts query
  }
}

Without DataLoader:
1. Query users → SELECT * FROM users
2. For each user (100 users):
   - Query posts → SELECT * FROM posts WHERE user_id = ?
   
Total: 101 queries!

With DataLoader:
1. Query users → SELECT * FROM users
2. Collect all user_ids
3. Batch query posts → SELECT * FROM posts WHERE user_id IN (...)
4. Distribute results to resolvers

Total: 2 queries!
"""

# Create DataLoaders
async def load_users(keys: List[int]) -> List[User]:
    """Batch load users by IDs."""
    async with AsyncSessionLocal() as db:
        result = await db.execute(
            select(UserModel).where(UserModel.id.in_(keys))
        )
        users = {u.id: u for u in result.scalars()}
        
        # Return in same order as keys (required by DataLoader)
        return [users.get(k) for k in keys]

async def load_posts_by_author(keys: List[int]) -> List[List[Post]]:
    """Batch load posts for multiple authors."""
    async with AsyncSessionLocal() as db:
        result = await db.execute(
            select(PostModel).where(PostModel.author_id.in_(keys))
        )
        
        # Group posts by author_id
        posts_by_author = defaultdict(list)
        for post in result.scalars():
            posts_by_author[post.author_id].append(post)
        
        # Return lists in key order
        return [posts_by_author.get(k, []) for k in keys]

# Initialize loaders per-request
def get_loaders():
    return {
        "user_loader": DataLoader(load_fn=load_users),
        "posts_by_author_loader": DataLoader(load_fn=load_posts_by_author),
    }

# Updated schema with DataLoader
@strawberry.type
class User:
    id: strawberry.ID
    username: str
    
    @strawberry.field
    async def posts(self, info) -> List[Post]:
        """Use DataLoader instead of direct DB query."""
        loader = info.context["loaders"]["posts_by_author_loader"]
        return await loader.load(int(self.id))

# Update context
async def get_graphql_context():
    async with AsyncSessionLocal() as db:
        yield {
            "db": db,
            "loaders": get_loaders(),  # Fresh loaders per request
            "user_id": get_current_user_id(),
        }
```

#### Authentication in GraphQL

```python
# graphql_auth.py - Authentication patterns
import strawberry
from strawberry.types import Info
from fastapi import Depends, HTTPException, status
from jose import jwt, JWTError

@strawberry.type
class AuthError:
    """Error type for auth failures."""
    message: str
    code: str

@strawberry.type
class AuthPayload:
    """Union type for auth results."""
    user: Optional[User] = None
    error: Optional[AuthError] = None
    token: Optional[str] = None

# Authentication dependency
async def get_current_user_id_from_token(info: Info) -> Optional[str]:
    """Extract user ID from JWT in request headers."""
    request = info.context["request"]
    
    auth_header = request.headers.get("authorization")
    if not auth_header or not auth_header.startswith("Bearer "):
        return None
    
    token = auth_header[7:]  # Remove "Bearer "
    
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload.get("sub")
    except JWTError:
        return None

# Protected resolver
@strawberry.type
class Mutation:
    @strawberry.mutation
    async def create_post(
        self,
        info: Info,
        title: str,
        content: str
    ) -> Post:
        """Create post - requires authentication."""
        user_id = await get_current_user_id_from_token(info)
        
        if not user_id:
            raise Exception("Authentication required")
        
        db = info.context["db"]
        post = await create_post_db(db, user_id, title, content)
        
        return Post(
            id=str(post.id),
            title=post.title,
            content=post.content,
            author_id=user_id
        )

# Alternative: Field-level permissions
def has_role(role: str):
    """Decorator for field-level permissions."""
    def decorator(resolver):
        async def wrapper(self, info: Info, *args, **kwargs):
            user_id = info.context.get("user_id")
            if not user_id:
                raise Exception("Not authenticated")
            
            # Check role in database
            db = info.context["db"]
            user = await get_user_by_id(db, user_id)
            if role not in user.roles:
                raise Exception(f"Required role: {role}")
            
            return await resolver(self, info, *args, **kwargs)
        return wrapper
    return decorator

@strawberry.type
class Query:
    @strawberry.field
    @has_role("admin")
    async def all_users(self, info: Info) -> List[User]:
        """Admin-only query."""
        db = info.context["db"]
        return await list_all_users(db)
```

#### Subscriptions (Real-time Updates)

```python
# subscriptions.py - Real-time GraphQL
import strawberry
from strawberry.subscriptions import GRAPHQL_TRANSPORT_WS_PROTOCOL
from typing import AsyncGenerator
import asyncio

@strawberry.type
class Subscription:
    @strawberry.subscription
    async def user_created(self) -> AsyncGenerator[User, None]:
        """
        Subscribe to new user creation.
        
        Yields users as they're created.
        """
        while True:
            # Wait for new user signal (from Redis pub/sub, etc.)
            user_data = await redis_subscribe("user_created")
            
            yield User(
                id=user_data["id"],
                username=user_data["username"],
                email=user_data["email"],
                created_at=user_data["created_at"]
            )
    
    @strawberry.subscription
    async def post_published(
        self,
        author_id: Optional[strawberry.ID] = None
    ) -> AsyncGenerator[Post, None]:
        """
        Subscribe to post publications.
        
        Optional filter by author_id.
        """
        channel = f"posts:{author_id}" if author_id else "posts:*"
        
        async for message in redis.psubscribe(channel):
            post_data = json.loads(message["data"])
            
            yield Post(
                id=post_data["id"],
                title=post_data["title"],
                content=post_data["content"],
                author_id=post_data["author_id"],
                published=True
            )

# Update schema
schema = strawberry.Schema(
    query=Query,
    mutation=Mutation,
    subscription=Subscription
)

# FastAPI router with WebSocket support
app.include_router(
    GraphQLRouter(
        schema,
        context_getter=get_graphql_context,
        subscription_protocols=[GRAPHQL_TRANSPORT_WS_PROTOCOL],
    ),
    prefix="/graphql"
)

# Triggering subscriptions from mutations
@strawberry.type
class Mutation:
    @strawberry.mutation
    async def create_post(self, info: Info, title: str, content: str) -> Post:
        db = info.context["db"]
        user_id = info.context["user_id"]
        
        post = await create_post_db(db, user_id, title, content)
        
        # Publish to subscribers
        await redis.publish(
            f"posts:{user_id}",
            json.dumps({
                "id": str(post.id),
                "title": post.title,
                "content": post.content,
                "author_id": user_id
            })
        )
        
        return post
```

#### Query Complexity Analysis

```python
# complexity.py - Preventing abusive queries
from strawberry.extensions import Extension
from strawberry.types import ExecutionContext
import strawberry

class QueryComplexityExtension(Extension):
    """
    Prevent expensive queries from overloading server.
    
    Analyzes query depth and field complexity before execution.
    """
    
    def on_request_start(self):
        self.complexity = 0
        self.max_complexity = 1000
        self.max_depth = 10
    
    def on_parsing_end(self, execution_context: ExecutionContext):
        """Calculate query complexity after parsing."""
        query = execution_context.query
        
        # Simple depth calculation
        depth = self._calculate_depth(query)
        if depth > self.max_depth:
            raise Exception(f"Query too deep: {depth} > {self.max_depth}")
        
        # Field complexity (simplified)
        self.complexity = self._calculate_complexity(query)
        if self.complexity > self.max_complexity:
            raise Exception(f"Query too complex: {self.complexity}")
    
    def _calculate_depth(self, query: str) -> int:
        """Calculate nesting depth."""
        max_depth = 0
        current_depth = 0
        
        for char in query:
            if char == '{':
                current_depth += 1
                max_depth = max(max_depth, current_depth)
            elif char == '}':
                current_depth -= 1
        
        return max_depth
    
    def _calculate_complexity(self, query: str) -> int:
        """Estimate query cost."""
        # Simple heuristic: count fields
        return query.count('{')

# Use in schema
schema = strawberry.Schema(
    query=Query,
    mutation=Mutation,
    extensions=[QueryComplexityExtension]
)
```

---

### Summary

In this chapter, you integrated GraphQL with FastAPI:

1. **GraphQL Fundamentals**: Understood the type system (Objects, Scalars, Enums), operations (Query, Mutation, Subscription), and the single-endpoint architecture that allows clients to request exactly the fields they need.

2. **Strawberry Integration**: Set up Strawberry for code-first schema definition using Python type hints, created type-safe resolvers with async support, and integrated with FastAPI using `GraphQLRouter` with context injection for database sessions and authentication.

3. **Advanced Patterns**: Implemented DataLoaders to solve the N+1 query problem through batch loading, added JWT-based authentication with field-level permission decorators, created real-time subscriptions using Redis pub/sub and WebSockets, and added query complexity analysis to prevent abusive requests.

**GraphQL Best Practices:**
- Use DataLoaders for all relationships to prevent N+1 queries
- Implement query complexity limits and depth restrictions
- Handle authentication at the context level, not per-resolver
- Use subscriptions sparingly (they maintain persistent connections)
- Cache DataLoader results per-request
- Log slow queries for optimization

---

### What's Next?

**Chapter 24: Advanced Patterns** will cover:
- **Server-Sent Events (SSE)**: Implementing unidirectional server-to-client streaming for real-time updates without WebSocket complexity
- **Custom OpenAPI**: Modifying the auto-generated OpenAPI schema for custom documentation, deprecated endpoints, and vendor extensions
- **Sub-Applications**: Mounting other ASGI applications (Django admin, Flask legacy routes) within FastAPI using `mount()` for gradual migration strategies

This final chapter completes your toolkit with advanced integration patterns for complex production scenarios.